# SD202 TP2 - Normalization and SQL

The objectives of this TP are the following:

1. Apply normalization 1NF -> 2NF -> 3NF
2. Perform SQL queries on the normalized database

In this lab, we are going to use a database containing wine information related to 'production' and 'sales'. 

Production <---> Wine <---> Sales


---

### Working with db files in Jupyter
- Python provides an interface for SQLite through the *sqlite3* module
- The **`%%sql`** magic builds upon it (and other tools) to enable the usage of SQL commands within a Jupyter Notebook as in common SQL clients.
- Before proceeding, make sure that **`wine.db`** is in the same path as this notebook.
  - If **`wine.db`** is not in the same path, an empty **`wine.db`** file will be created, resulting in errors in later steps of the TP.
- The SQLite module in Python commits transactions automatically, this means that any change in the DB is immediately written to the file, e.g. creating/deleting tables.
  -  For this reason, it is recommended to have a backup of **`wine.db`** as it is provided in the course's website.

---

**`wine.db`** contains the following unnormalized tables:

<center>**Master1**</center>

|*Attribute*|         *Description*          |
| -------   |--------------------------------|
| NV        | Wine number                    |
| CRU       | Vineyard or group of vineyards |
| DEGRE     | Alcohol content                |
| MILL      | Vintage year                   |
| QTE       | Number of bottles harvested    |
| NP        | Producer number                |
| NOM       | Producer's last name           |
| PRENOM    | Producer's first name          |
| REGION    | Production region              |

From wikipedia:

__Cru__: Often used to indicate a specifically named and legally defined vineyard or ensemble of vineyards and the vines "which grow on [such] a reputed terroir; by extension of good quality." The term is also used to refer to the wine produced from such vines.


<center>**Master2**</center>

|*Attribute*|                         *Description*                  |
| -------   |--------------------------------------------------------|
| NV        | Wine number                                            |
| CRU       | Vineyard or group of vineyards                         |
| DEGRE     | Alcohol content                                        |
| MILL      | Vintage year                                           |
| DATES     | Buying date                                            |
| LIEU      | Place where the wine was sold                          |
| QTE       | Number of bottles bought                               |
| NB        | Client (buveur) number                                 |
| NOM       | Client's last name                                     |
| PRENOM    | Client's first name                                    |
| TYPE      | Type of client by volume of purchases                  |
| REGION    | Administrative Region (different to production region) |


We need to prepare the SQL environment:

In [2]:
import sqlite3

In [3]:
def printSchema(connection):
    ### Source: http://stackoverflow.com/a/35092773/4765776
    for (tableName,) in connection.execute(
        """
        select NAME from SQLITE_MASTER where TYPE='table' order by NAME;
        """
    ):
        print("{}:".format(tableName))
        for (
            columnID, columnName, columnType,
            columnNotNull, columnDefault, columnPK,
        ) in connection.execute("pragma table_info('{}');".format(tableName)):
            print("  {id}: {name}({type}){null}{default}{pk}".format(
                id=columnID,
                name=columnName,
                type=columnType,
                null=" not null" if columnNotNull else "",
                default=" [{}]".format(columnDefault) if columnDefault else "",
                pk=" *{}".format(columnPK) if columnPK else "",
            ))

In [4]:
conn = sqlite3.connect('wine.db')
c = conn.cursor()
print("Database schema:")
printSchema(conn)           # An usefull way to viualize the content of the database

Database schema:
Lieu:
  0: LIEU(TEXT)
  1: REGION(TEXT)
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(NUM)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
Product1:
  0: NV(NUM)
  1: NP(NUM)
  2: QTE(NUM)
Product2:
  0: NV(NUM)
  1: NB(NUM)
  2: DATES(NUM)
  3: LIEU(TEXT)
  4: QTE(NUM)
Producter1:
  0: NP(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: REGION(TEXT)
Producter2:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
Vin1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
Vin2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)


We recommend inline __%sql__ as an alternative to sqlite3 package

In [5]:
!pip3 install ipython-sql
%load_ext sql
%sql sqlite:///wine.db

Now, we can see the content of the tables using SQL queries:

In [7]:
%sql SELECT NV, NP, CRU, MILL, DEGRE FROM MASTER1;

 * sqlite:///wine.db
Done.


NV,NP,CRU,MILL,DEGRE
None,3,None,None,None
None,6,None,None,None
None,8,None,None,None
None,16,None,None,None
None,17,None,None,None
None,18,None,None,None
None,19,None,None,None
None,20,None,None,None
None,21,None,None,None
None,26,None,None,None


# PART I: Database normalization

The first task on this TP is the normalization of the wine data. In its current state both tables Master1 and Master2 are in the First Normal Form (1NF) and suffer from data redundancy. 

__1.1__ Given the set of the following FDs:

NV -> CRU, DEGRE, MILL

NP -> NOM, PRENOM, REGION

NV, NP -> QTE

Convert table Master1 to the Second Normal Form (2NF) or Third Normal Form (3NF) as applicable.

* Explain your answer
* Describe the schema of new tables and how they relate

__1.2__ Given the set of the following FDs:

NV -> CRU, DEGRE, MILL

NB -> NOM, PRENOM, TYPE

NV, NB -> DATES, LIEU, QTE

LIEU -> REGION 

Convert table Master2 to the Second Normal Form (2NF) or Third Normal Form (3NF) as applicable. 
* Explain your answer
* Describe the schema of new tables and how they relate

Once you have defined the 2NF or 3NF (as applicable) we need to split the data into new tables. A table can be created from the result of a query. In the following example we will create a new table "dummy" to store the different values of alcohol content.

In [13]:
%sql DROP TABLE IF EXISTS dummy;

# Create dummy table
%sql CREATE TABLE dummy AS \
SELECT DISTINCT DEGRE \
FROM MASTER1;

print("\nContent of the database")
printSchema(conn)

print("\nContent of dummy")
%sql SELECT * FROM dummy

 * sqlite:///wine.db
Done.
 * sqlite:///wine.db
Done.

Content of the database
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(NUM)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
dummy:
  0: DEGRE(NUM)

Content of dummy
 * sqlite:///wine.db
Done.


DEGRE
None
11.5
11.3
12.1
10.9
11.7
11.2
12.3
11.9
11.8


In [17]:
# Remove dummy table
%sql DROP TABLE IF EXISTS dummy;

%sql DROP TABLE IF EXISTS Product1;
%sql DROP TABLE IF EXISTS Vin1;
%sql DROP TABLE IF EXISTS Producter1;

 * sqlite:///wine.db
Done.
 * sqlite:///wine.db
Done.
 * sqlite:///wine.db
Done.
 * sqlite:///wine.db
Done.


[]

__1.3__ Create the new tables from Master1:

In [18]:
%sql CREATE TABLE Product1 AS SELECT Distinct NV, NP, QTE FROM Master1;

%sql CREATE TABLE Vin1 AS SELECT distinct NV, Cru, degre, mill FROM Master1;

%sql CREATE TABLE Producter1 AS SELECT distinct NP, nom, prenom, region FROM Master1;

 * sqlite:///wine.db
Done.
 * sqlite:///wine.db
Done.
 * sqlite:///wine.db
Done.


[]

__1.4__ Create the new tables from Master2:

In [20]:
%sql CREATE TABLE Product2 AS SELECT Distinct NV, NB, Dates, Lieu, Qte FROM Master2;

%sql CREATE TABLE Vin2 AS SELECT DISTINCT NV, CRU, DEGRE, MILL FROM Master2;

%sql CREATE TABLE Producter2 AS SELECT DISTINCT NB, Nom, Prenom, Type FROM Master2;

%sql CREATE TABLE Lieu AS SELECT Distinct Lieu, Region FROM Master2;

 * sqlite:///wine.db
Done.
 * sqlite:///wine.db
(sqlite3.OperationalError) table Vin2 already exists
[SQL: CREATE TABLE Vin2 AS SELECT DISTINCT NV, CRU, DEGRE, MILL FROM Master2;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
 * sqlite:///wine.db
(sqlite3.OperationalError) table Producter2 already exists
[SQL: CREATE TABLE Producter2 AS SELECT DISTINCT NB, Nom, Prenom, Type FROM Master2;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
 * sqlite:///wine.db
(sqlite3.OperationalError) table Lieu already exists
[SQL: CREATE TABLE Lieu AS SELECT Distinct Lieu, Region FROM Master2;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


# PART II: SQL QUERIES

In the second part of this TP you will create SQL queries to retrieve information from the database.

__2.1__ What are the different types of clients (buveurs) by volume of purchases?

In [23]:
%sql SELECT DISTINCT TYPE AS Type_of_buveurs FROM Producter2 WHERE TYPE IS NOT NULL; 

 * sqlite:///wine.db
Done.


Type_of_buveurs
petit
moyen
gros


__2.2__ What regions produce Pommard or Brouilly?

In [25]:
%sql SELECT DISTINCT Pr.region FROM Producter1 Pr JOIN Product1 P ON Pr.NP = P.NP JOIN VIN1 V ON P.NV = V.NV WHERE V.Cru IN ('Pommard', 'Brouilly');

 * sqlite:///wine.db
Done.


REGION
Bourgogne
Rhone


__2.3__ What regions produce Pommard and Brouilly?

In [8]:
%sql SELECT DISTINCT Pr.region FROM Producter1 Pr JOIN Product1 P ON Pr.NP = P.NP JOIN VIN1 V ON P.NV = V.NV WHERE V.Cru IN ('Pommard', 'Brouilly') GROUP BY REGION HAVING COUNT(CRU)=2  ;

 * sqlite:///wine.db
Done.


REGION
Bourgogne


__2.4__ Get the number of wines bught by CRU and Millésime

In [9]:
%sql SELECT CRU, MILL, SUM(QTE) Number_of_wines_bought FROM Vin2 v1 JOIN Product2 v2 ON v1.NV=v2.NV WHERE CRU IS NOT NULL AND MILL IS NOT NULL AND QTE IS NOT NULL GROUP BY CRU, MILL;

 * sqlite:///wine.db
Done.


CRU,MILL,Number_of_wines_bought
Arbois,1980,8
Auxey Duresses,1914,80
Beaujolais Primeur,1983,7
Beaujolais Villages,1975,10
Beaujolais Villages,1976,120
Beaujolais Villages,1978,130
Beaujolais Villages,1979,520
Chapelle Chambertin,1973,30
Chateau Corton Grancey,1980,4
Chenas,1984,1


__2.5__ Retrieve the wine number (NV) of wines produced by more than three producers

In [10]:
%sql SELECT NV, COUNT(NP) Number_of_producers FROM Product1 WHERE NV IS NOT NULL GROUP BY NV HAVING COUNT(NP) > 3;

 * sqlite:///wine.db
Done.


NV,Number_of_producers
45,5
78,5
89,4
98,5


__2.6__ Which producers have not produced any wine?

In [12]:
%sql SELECT pr.NP, pr.NOM, pr.PRENOM FROM Producter1 pr JOIN Product1 p ON pr.NP = p.NP WHERE p.QTE IS NULL;

 * sqlite:///wine.db
Done.


NP,NOM,PRENOM
3,Six,Paul
6,Marmagne,Bernard
8,Lioger d'Harduy,Gabriel
16,Barbin,Bernard
17,Faiveley,Guy
18,Tramier,Jean
19,Dupaquier,Roger
20,Lamy,Jean
21,Cornu,Edmond
26,Violot,Gilbert


__2.10__ List all wines from 1980

In [13]:
%sql SELECT NV, CRU, MILL, DEGRE FROM Vin2 WHERE MILL=1980;

 * sqlite:///wine.db
Done.


NV,CRU,MILL,DEGRE
1,Mercurey,1980,11.5
4,Mercurey,1980,10.9
16,Meursault,1980,12.1
20,Cote de Brouilly,1980,12.1
26,Chateau Corton Grancey,1980,None
28,Volnay,1980,11
43,Fleurie,1980,11.4
74,Arbois,1980,12
78,Etoile,1980,12
79,Seyssel,1980,11


__2.11__ What are the wines from 1980 bought by NB=2?

In [15]:
%sql SELECT vi.NV, p.NB, p.DATES, p.LIEU, p.QTE, vi.CRU, vi.MILL, vi.DEGRE FROM Vin2 vi JOIN Product2 p ON vi.NV = p.NV WHERE vi.MILL=1980 AND p.NB=2 ;

 * sqlite:///wine.db
Done.


NV,NB,DATES,LIEU,QTE,CRU,MILL,DEGRE
1,2,1977-11-02,BORDEAUX,33,Mercurey,1980,11.5
